In [0]:
#Imports
import pandas as pd
import numpy as np
import tensorflow as tf
# tf.enable_eager_execution()
import os
import time

In [2]:
# Read, then decode for py2 compat.
path_to_file = tf.keras.utils.get_file('donald-tweets-clean.txt', 'https://github.com/Varun-S-Nair/Trump-Tweet-Generator/raw/master/RNNBackend/donald-tweets-clean.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 291204 characters


In [3]:
# Take a look at the first 250 characters in text
# text = text[5:]
print(text[:250])

Busy day planned in New York. Will soon be making some very important decisions on the people who will be running our government!
Love the fact that the small groups of protesters last night have passion for our great country. We will all come togeth


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

84 unique characters


In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '#' :   3,
  '$' :   4,
  '%' :   5,
  '&' :   6,
  '(' :   7,
  ')' :   8,
  '+' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
}


In [6]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Busy day plan' ---- characters mapped to int ---- > [29 75 73 79  1 58 55 79  1 70 66 55 68]


In [7]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

B
u
s
y
 


In [8]:
#The batch method lets us easily convert these individual characters to sequences of the desired size.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Busy day planned in New York. Will soon be making some very important decisions on the people who will be running our government!\nLove the fact that th'
'e small groups of protesters last night have passion for our great country. We will all come together and be proud!\nJust had a very open and successful'
' presidential election. Now professional protesters, incited by the media, are protesting. Very unfair!\nA fantastic day in D.C. Met with President Obam'
'a for first time. Really good meeting, great chemistry. Melania liked Mrs. O a lot!\nSuch a beautiful and important evening! The forgotten man and woman'
' will never be forgotten again. We will all come together as never before\nDont let up, keep getting out to vote - this election is FAR FROM OVER! We ar'


In [9]:
#For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a 
#simple function to each batch:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

#Print the first examples input and target values:

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Busy day planned in New York. Will soon be making some very important decisions on the people who will be running our government!\nLove the fact that t'
Target data: 'usy day planned in New York. Will soon be making some very important decisions on the people who will be running our government!\nLove the fact that th'


In [10]:
#Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the 
#index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same 
#thing but the RNN considers the previous step context in addition to the current input character.

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 29 ('B')
  expected output: 75 ('u')
Step    1
  input: 75 ('u')
  expected output: 73 ('s')
Step    2
  input: 73 ('s')
  expected output: 79 ('y')
Step    3
  input: 79 ('y')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 58 ('d')


In [11]:
#Create training batches
#We used tf.data to split the text into manageable sequences. But before feeding this data into the model, 
#we need to shuffle the data and pack it into batches.

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [15]:
#First check the shape of the output:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 150, 84) # (batch_size, sequence_length, vocab_size)


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           21504     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 84)            86100     
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [17]:
#To get actual predictions from the model we need to sample from the output distribution, to get actual character 
#indices. This distribution is defined by the logits over the character vocabulary.

#Try it for the first example in the batch:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([58, 19, 15, 83, 30, 26, 83, 34, 66, 17, 80, 49, 47,  4, 80, 44, 29,
       53, 28, 37, 30,  1, 65, 20, 41, 76, 67, 47, 76, 18, 83, 63,  1, 76,
       26, 27,  9, 72, 81, 78, 61, 38, 59, 31, 11, 65, 72, 79, 52, 76, 21,
       75,  5, 43, 54, 31, 35, 78, 71, 22, 57, 26, 21, 44, 71,  7, 36, 21,
        2, 77, 36, 53, 60, 35, 75,  2, 46, 13, 30,  4, 61, 31, 15,  0, 68,
       66, 44, 77, 51, 40,  4, 45, 26, 31,  5, 39, 80,  1, 59,  0, 62, 12,
       74, 39,  6, 68, 21, 77, 21, 55, 11, 64, 82, 79, 58, 57, 81, 59, 37,
       43, 11, 30,  5, 67, 39, 64, 51, 62, 15, 74, 22, 34, 47, 15, 16, 71,
       51, 62, 14, 58, 17, 16, 63, 64, 34, 28, 58, 64, 40, 19])

In [18]:
#Decode these to see the text predicted by this untrained model:

print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ent with great leadership skills and vision, not someone like Hillary or Barack, neither of which has a clue!\nThe phony Club For Growth, which asked m'

Next Char Predictions: 
 'd51۪C?۪Gl3zVT$zQBZAJC k6NvmTv4۪i v?@+ṟxgKeD-kryYv7u%P_DHxq8c?7Qq(I7!wIZfHu!S/C$gD1\nnlQwXM$R?D%Lz e\nh.tL&n7w7a-j\u06ddydc̱eJP-C%mLjXh1t8GT12qXh0d32ijGAdjM5'


In [19]:
#Train the Model
#At this point the problem can be treated as a standard classification problem. Given the previous RNN state, 
#and the input this time step, predict the class of the next character.

#Attach an optimizer, and a loss function
#The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is 
#applied across the last dimension of the predictions.

#Because our model returns logits, we need to set the from_logits flag.

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 150, 84)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.43088


In [0]:
#Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with 
#default arguments and the loss function.

model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [0]:
#Configure checkpoints
#Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:
# Directory where the checkpoints will be saved
!rm -rf ./training_checkpoints
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [22]:
#Execute the training
#To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster 
#training.
#Loss minimized at epoch 72 (0.3808), Accuracy maximized at epoch 57/66/70
EPOCHS=72

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/72
30/30 [==============================] - 3s 86ms/step - loss: 3.8055 - accuracy: 0.1276
Epoch 2/72
30/30 [==============================] - 3s 85ms/step - loss: 2.9421 - accuracy: 0.2358
Epoch 3/72
30/30 [==============================] - 3s 87ms/step - loss: 2.6174 - accuracy: 0.2903
Epoch 4/72
30/30 [==============================] - 3s 88ms/step - loss: 2.4740 - accuracy: 0.3139
Epoch 5/72
30/30 [==============================] - 3s 88ms/step - loss: 2.3764 - accuracy: 0.3378
Epoch 6/72
30/30 [==============================] - 3s 89ms/step - loss: 2.2744 - accuracy: 0.3606
Epoch 7/72
30/30 [==============================] - 3s 89ms/step - loss: 2.1573 - accuracy: 0.3940
Epoch 8/72
30/30 [==============================] - 3s 90ms/step - loss: 2.0398 - accuracy: 0.4276
Epoch 9/72
30/30 [==============================] - 3s 87ms/step - loss: 1.9267 - accuracy: 0.4601
Epoch 10/72
30/30 [==============================] - 3s 88ms/step - loss: 1.8199 - accuracy: 0.4906
Epoch 11/

In [23]:
#Generate text
#Restore the latest checkpoint
#To keep this prediction step simple, use a batch size of 1.

#Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size 
#once built.

#To run the model with a different batch_size, we need to rebuild the model and restore the weights from the 
#checkpoint.

tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_72'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            21504     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86100     
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [26]:
#The prediction loop
#The following code block generates the text:

#It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

#Get the prediction distribution of the next character using the start string and the RNN state.

#Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character 
#as our next input to the model.

#The RNN state returned by the model is fed back into the model so that it now has more context, instead than only 
#one character. After predicting the next character, the modified RNN states are again fed back into the model, 
#which is how it learns as it gets more context from the previously predicted characters.

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"M"))

#The easiest thing you can do to improve the results it to train it for longer (try EPOCHS=30).

#You can also experiment with a different start string, or try adding another RNN layer to improve the model's 
#accuracy, or adjusting the temperature parameter to generate more or less random predictions.

MERICA very dumb funny, for your kind words. We will have a do why Mitt Romney Caniflinto and she went so important--- stopping illegal immigration!
Just got great national poll numbers. We got she hoselergust polls against me. She is a joke!
I am not just running against Crooked Hillary Clinton doesnt have the strength or storm!
She doesnt even look presidential!
Crooked Hillary no longer has crowd!
Great Trudpan control oupport!
Watching John Kasich believe that mach that mession- he doesnt have a clue. I hear she will soon be gone!
.@secupp, who cant believe that her husband should not be allowed in in fact. Nothonest media!
I am at Trump National Doral-best resort in newspice any of very sad!
Thank you Kayle Hillary Clinton cannot even bring herself to say he wants to run against me.
Senator Liberty Instruct. See you there.
That is happy with a chekeand that their was no ISIS vidio and he U.S. Senate. She has done nothing!
I dont want to hit Crazing @megynkelly Stupid about them th

In [0]:
#Advanced: Customized Training
#The above training procedure is simple, but does not give you much control.

#So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. 
#This gives a starting point if, for example, to implement curriculum learning to help stabilize the model's open-loop 
#output.

#We will use tf.GradientTape to track the gradients. You can learn more about this approach by reading the eager 
#execution guide.

#The procedure works as follows:

#First, initialize the RNN state. We do this by calling the tf.keras.Model.reset_states method.

#Next, iterate over the dataset (batch by batch) and calculate the predictions associated with each.

#Open a tf.GradientTape, and calculate the predictions and loss in that context.

#Calculate the gradients of the loss with respect to the model variables using the tf.GradientTape.grads method.

#Finally, take a step downwards by using the optimizer's tf.train.Optimizer.apply_gradients method.

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [29]:
# Training step
EPOCHS = 72

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.432528972625732
Epoch 1 Loss 3.1484
Time taken for 1 epoch 3.853494644165039 sec

Epoch 2 Batch 0 Loss 3.1502151489257812
Epoch 2 Loss 2.7118
Time taken for 1 epoch 2.667045831680298 sec

Epoch 3 Batch 0 Loss 2.7216367721557617
Epoch 3 Loss 2.4943
Time taken for 1 epoch 2.6748578548431396 sec

Epoch 4 Batch 0 Loss 2.5583674907684326
Epoch 4 Loss 2.4216
Time taken for 1 epoch 2.685960292816162 sec

Epoch 5 Batch 0 Loss 2.4174256324768066
Epoch 5 Loss 2.2866
Time taken for 1 epoch 2.726372003555298 sec

Epoch 6 Batch 0 Loss 2.2953503131866455
Epoch 6 Loss 2.1820
Time taken for 1 epoch 2.695415735244751 sec

Epoch 7 Batch 0 Loss 2.197901725769043
Epoch 7 Loss 2.0779
Time taken for 1 epoch 2.674403429031372 sec

Epoch 8 Batch 0 Loss 2.0747458934783936
Epoch 8 Loss 1.8986
Time taken for 1 epoch 2.7174761295318604 sec

Epoch 9 Batch 0 Loss 1.8992677927017212
Epoch 9 Loss 1.8610
Time taken for 1 epoch 2.7082982063293457 sec

Epoch 10 Batch 0 Loss 1.8210840225219727
Epoc

In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_71'

In [31]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            21504     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 84)             86100     
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [32]:
print(generate_text(model, start_string=u"S"))

Ston the other candidates.
Just another candidates are finally  should have know that the Republicans picked Cleveland insthat Iowa Cauco dumb us all of the at four MAKE AMERICA GREAT AGAIN!
The endorsement of me by the 11,500 Bold, Jers. How many tale-lied to the FBI and to the people of New York, and all others laughing!
The ratings for the Republican Party that are currently and selfishly opposed to the Fadellary Wadly, they are not about me run against Crooked Hillary Clinton than protected to explain away the new Monmouth poll 41 to 14 of the media hit job. Leading Ohio 48 - 44.
Voter Trump University case but the preds where I have a waste of time being interviewed by @FoxNews was a disgrace to good broadcasting and journalism. Who would ever had, to announce this Salama Sanders is her way up and people who control our politicians (puppets) are spending the way for many great Supreme Cooright on @FoxNews Tony. I am with you all the way! Make America Great Again!
Rany people have 